In [ ]:
pip install librosa


In [2]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from scipy import signal
import librosa
from IPython.display import Audio


In [13]:
### 5.1 Surround downmixer to Stereo 
# importing 5.1 file 

vss_Center, sr = librosa.load('Center.wav', sr = None, mono = True)
vss_Front_left, sr = librosa.load('FrontLeft.wav', sr = None, mono = True)
vss_Front_right, sr = librosa.load('FrontRight.wav', sr = None, mono = True)
vss_Rear_left, sr = librosa.load('RearLeft.wav', sr = None, mono = True)
vss_Rear_right, sr = librosa.load('RearRight.wav',sr = None, mono = True)











In [12]:
# importing HRIRs at different azimouths
# dont need to split the HRIRs to left and right mono, keep them in stereo, 
# jsut send left azimouth to left hrir and the same for the right 
sig_Center, sr = sf.read('IRC_1002_C_R0195_T000_P000.wav')
# split to l and r binaural 
sig_L_Center = sig_Center[:,0]
sig_R_Center = sig_Center[:,1]

sig_Right, sr = sf.read('IRC_1002_C_R0195_T030_P000.wav')
sig_L_Right = sig_Center[:,0]
sig_R_Right = sig_Center[:,1]

sig_Left, sr = sf.read('IRC_1002_C_R0195_T330_P000.wav')
sig_L_Left = sig_Left[:,0]
sig_R_Left = sig_Left[:,1]

sig_Right_Back, sr = sf.read('IRC_1002_C_R0195_T120_P000.wav')
sig_L_Right_Back = sig_Right_Back [:,0]
sig_R_Right_Back = sig_Right_Back [:,1]

sig_Left_Back, sr = sf.read('IRC_1002_C_R0195_T240_P000.wav')
sig_L_Left_Back = sig_Left_Back [:,0]
sig_R_Left_Back = sig_Left_Back [:,1]






In [15]:
# check if Left binaural and Right binaural are indeed different                      
def are_these_the_same(Left,Right):

    Truth = True 

    for i in range(len(Left)):
      if Left[i] != Right[i]:
         Truth = False

    
    return Truth 


In [16]:
#resize files if needed

vss_Front_left = np.resize(vss_Front_left,279437)









print(len(vss_Center))
print(len(vss_Front_left))

279437
279437


In [17]:
# zero padding function 
def zero_pad(input_sig,ir_sig):

  if len(input_sig) > len(ir_sig):
    output = np.hstack([ir_sig, np.zeros((len(input_sig)-len(ir_sig)))])


  return output
  

In [19]:


###################

zero_pad_Center_L=zero_pad(vss_Center,sig_L_Center)
zero_pad_Center_R=zero_pad(vss_Center,sig_R_Center)
zero_pad_Front_Left_L=zero_pad(vss_Front_left,sig_L_Left)
zero_pad_Front_Left_R=zero_pad(vss_Front_left,sig_R_Left)
zero_pad_Front_right_L=zero_pad(vss_Front_right,sig_L_Right)
zero_pad_Front_right_R=zero_pad(vss_Front_right,sig_R_Right)
zero_pad_Rear_Left_L=zero_pad(vss_Rear_left,sig_L_Left_Back)
zero_pad_Rear_Left_R=zero_pad(vss_Rear_left,sig_R_Left_Back)
zero_pad_Rear_Right_L=zero_pad(vss_Rear_right,sig_L_Right_Back)
zero_pad_Rear_Right_R=zero_pad(vss_Rear_right,sig_R_Right_Back)

zero_pad_cycle = [zero_pad_Center_L, 
                    zero_pad_Center_R, 
                    zero_pad_Front_Left_L, 
                    zero_pad_Front_Left_R,
                    zero_pad_Front_right_L,
                    zero_pad_Front_right_R,zero_pad_Rear_Left_L,zero_pad_Rear_Left_R,zero_pad_Rear_Right_L,zero_pad_Rear_Right_R]

# ir_cycle = [sig_L_Center,sig_R_Center,sig_L_Left,sig_R_Left,sig_L_Right,sig_R_Right,sig_L_Left_Back,sig_R_Left_Back,sig_L_Right_Back,sig_R_Right_Back]
#             0.         ,1.           ,2.        ,3.        ,4.        ,5.         ,6.             ,7.             ,8.              ,9
              
                    
print(len(zero_pad_cycle))
  




10


In [20]:
def convolve(a,b):
  for i in range(0,len(zero_pad_cycle)):
    if i < len(zero_pad_cycle):
       z = zero_pad(a,b)

    output = np.real(np.fft.ifft(np.fft.fft(a) * np.fft.fft(z)))

  
  return output 


In [22]:
#we have 5 arrays with proccesed channels, sum to stereo downmix using soundfile.write 

#LEFT
center_left_convolve = convolve(vss_Center,sig_L_Center) 


right_left_convolve = .707 * convolve(vss_Front_right,sig_L_Right) 

left_left_convolve = convolve(vss_Front_left,sig_L_Left) 

rightback_left_convolve = convolve(vss_Rear_right,sig_L_Right_Back)

leftback_left_convolve = convolve(vss_Rear_left, sig_L_Left_Back)

Left_Binaural_Final_Sum = center_left_convolve + right_left_convolve + left_left_convolve + rightback_left_convolve + leftback_left_convolve

#RIGHT
center_right_convolve = .707 * convolve(vss_Center,sig_R_Center) 

right_right_convolve = convolve(vss_Front_right,sig_R_Right) 

left_right_convolve = convolve(vss_Front_left,sig_R_Left) 

rightback_right_convolve = convolve(vss_Rear_right,sig_R_Right_Back)

leftback_right_convolve = convolve(vss_Rear_left, sig_R_Left_Back)

Right_Binaural_Final_Sum = center_right_convolve + right_right_convolve + left_right_convolve + rightback_right_convolve + leftback_right_convolve

#FINAL STEREO 
Final_Stereo_Downmix = [Left_Binaural_Final_Sum,Right_Binaural_Final_Sum]


print(Right_Binaural_Final_Sum)
Audio(Final_Stereo_Downmix, rate =44100)


[ 1.41410621e-04  8.32452627e-06  5.15838082e-04 ... -3.89881179e-04
  2.11712663e-04  1.05338969e-03]


In [23]:
#export file VVS proccesing 
sf.write(data=np.array(Final_Stereo_Downmix).T,file='Wang_Max_VSS_Downmix_FINAL.wav',samplerate=44100)


In [25]:
#Downmix no VVS proccesing 

Left_Stereo = .707*vss_Center + vss_Front_left + vss_Rear_left
Right_Stereo =.707*vss_Center + vss_Front_right + vss_Rear_right
Final_Stereo_No_VSS = [Left_Stereo,Right_Stereo]

Audio(Final_Stereo_No_VSS, rate = 44100)




In [ ]:
#export file no VSS 
sf.write(data=np.array(Final_Stereo_No_VSS).T,file='Wang_Max_No_VSS_Downmix_FINAL.wav',format= None ,samplerate=44100)